# Implement the closed form greeks for GBSM using python code. 

In [1]:
from scipy.stats import norm
import numpy as np
# Define the parameters for the option
current_stock_price = 151.03
strike_price = 165
risk_free_rate = 0.0425
continuously_compounding_coupon = 0.0053
current_date = np.datetime64('2022-03-13')
expiration_date = np.datetime64('2022-04-15')
time_to_expiration = (expiration_date - current_date).astype('timedelta64[D]').astype(int) / 365.0
import numpy as np
import pandas as pd
import os
from datetime import datetime
from functools import partial
from math import log, sqrt, exp, isclose
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import norm, t
import scipy

# Constants
current_date = datetime(2022, 3, 13)
expire_date = datetime(2022, 4, 15)
T = (expire_date - current_date).days / 365
S = 151.03
X = 165
implied_vol = 0.2
r = 0.0425
coupon = 0.0053
b = r - coupon

# Functions
def d1(S, X, T, implied_vol, b):
    return (np.log(S / X) + (b + implied_vol ** 2 / 2) * T) / (implied_vol * np.sqrt(T))

def d2(d1, T, implied_vol):
    return d1 - implied_vol * np.sqrt(T)

def greeks(option_type, S, X, T, implied_vol, r, b):
    call = 1 if option_type == "Call" else -1
    d_1 = d1(S, X, T, implied_vol, b)
    d_2 = d2(d_1, T, implied_vol)

    delta = call * norm.cdf(call * d_1, 0, 1)
    gamma = norm.pdf(d_1, 0, 1) / (S * implied_vol * np.sqrt(T))
    vega = S * norm.pdf(d_1, 0, 1) * np.sqrt(T)
    theta = -S * np.exp((b - r) * T) * norm.pdf(d_1, 0, 1) * implied_vol / (2 * np.sqrt(T)) \
            - (b - r) * S * np.exp((b - r) * T) * norm.cdf(call * d_1, 0, 1) * call \
            - r * X * np.exp(-r * T) * norm.cdf(call * d_2, 0, 1) * call
    rho = X * T * np.exp(-r * T) * norm.cdf(call * d_2, 0, 1) * call
    carry_rho = S * T * np.exp((b - r) * T) * norm.cdf(call * d_1, 0, 1) * call

    return delta, gamma, vega, theta, rho, carry_rho

option_types = ["Call", "Put"]
greeks_values = [greeks(option_type, S, X, T, implied_vol, r, b) for option_type in option_types]
delta, gamma, vega, theta, rho, carry_rho = zip(*greeks_values)
for i, option_type in enumerate(option_types):
    print(f"{option_type} Greeks:")
    print(f"  Delta: {delta[i]}")
    print(f"  Gamma: {gamma[i]}")
    print(f"  Vega: {vega[i]}")
    print(f"  Theta: {theta[i]}")
    print(f"  Rho: {rho[i]}")
    print(f"  Carry Rho: {carry_rho[i]}\n")

# Assume volatility is known, let's use an arbitrary volatility for demonstration purposes
# Placeholder for volatility, this would usually be implied volatility

# Calculate the Greeks for both call and put options
#greeks = black_scholes_greeks(current_stock_price, strike_price, time_to_expiration, risk_free_rate, continuously_compounding_coupon, volatility)

# Print the results
#for greek in greeks:
#    print(f"{greek}: {greeks[greek]}")

Call Greeks:
  Delta: 0.08301107089626869
  Gamma: 0.016830979206204362
  Vega: 6.942036604441163
  Theta: -8.126522359668838
  Rho: 1.1025939156368187
  Carry Rho: 1.132953825011723

Put Greeks:
  Delta: -0.9169889291037313
  Gamma: 0.016830979206204362
  Vega: 6.942036604441163
  Theta: -1.9409914783019566
  Rho: -13.758003122735788
  Carry Rho: -12.515271800549371



# Implement a finite difference derivative calculation

In [2]:
from scipy.stats import norm
import numpy as np

# Define the parameters for the option
current_stock_price = 151.03
strike_price = 165
risk_free_rate = 0.0425
continuously_compounding_coupon = 0.0053
current_date = np.datetime64('2022-03-13')
expiration_date = np.datetime64('2022-04-15')
volatility = 0.20  
time_to_expiration = (expiration_date - current_date).astype('timedelta64[D]').astype(int) / 365

import numpy as np
from scipy.stats import norm
from datetime import datetime
import inspect

# Constants
current_date = datetime(2022, 3, 13)
expire_date = datetime(2022, 4, 15)
T = (expire_date - current_date).days / 365
S = 151.03
X = 165
implied_vol = 0.2
r = 0.0425
coupon = 0.0053
b = r - coupon

# Functions
def d1(S, X, T, implied_vol, b):
    return (np.log(S / X) + (b + implied_vol ** 2 / 2) * T) / (implied_vol * np.sqrt(T))

def d2(d1, T, implied_vol):
    return d1 - implied_vol * np.sqrt(T)

def gbsm(option_type, S, X, T, implied_vol, r, b):
    call = 1 if option_type == "Call" else -1
    d_1 = d1(S, X, T, implied_vol, b)
    d_2 = d2(d_1, T, implied_vol)

    return call * (S * np.exp((b - r) * T) * norm.cdf(call * d_1, 0, 1) - X * np.exp(-r * T) * norm.cdf(call * d_2, 0, 1))

# Calculate first order derivative
def first_order_der(func, x, delta):
    return (func(x + delta) - func(x - delta)) / (2 * delta)

# Calculate second order derivative
def second_order_der(func, x, delta):
    return (func(x + delta) + func(x - delta) - 2 * func(x)) / delta ** 2

def cal_partial_derivative(func, order, arg_name, delta=1e-3):
    arg_names = list(inspect.signature(func).parameters.keys())
    derivative_fs = {1: first_order_der, 2: second_order_der}

    def partial_derivative(*args, **kwargs):
        args_dict = dict(list(zip(arg_names, args)) + list(kwargs.items()))
        arg_val = args_dict.pop(arg_name)

        def partial_f(x):
            p_kwargs = {arg_name: x, **args_dict}
            return func(**p_kwargs)

        return derivative_fs[order](partial_f, arg_val, delta)

    return partial_derivative

# Black-Scholes formula for option price
def black_scholes_price(S, K, T, r, q, sigma, option_type='call'):
    D1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    D2 = D1 - sigma * np.sqrt(T)
    if option_type == 'call':
        price = S * np.exp(-q * T) * norm.cdf(D1) - K * np.exp(-r * T) * norm.cdf(D2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-D2) - S * np.exp(-q * T) * norm.cdf(-D1)
    return price

# Define volatility, which would be estimated from market data (implied volatility)
volatility = 0.20  # Placeholder for volatility

option_types = ["Call", "Put"]
greek_names = ['Delta', 'Gamma', 'Vega', 'Theta', 'Rho', 'Carry Rho']
greek_funcs = {
    'Delta': cal_partial_derivative(gbsm, 1, 'S'),
    'Gamma': cal_partial_derivative(gbsm, 2, 'S'),
    'Vega': cal_partial_derivative(gbsm, 1, 'implied_vol'),
    'Theta': cal_partial_derivative(gbsm, 1, 'T', delta=-1e-6),
    'Rho': cal_partial_derivative(gbsm, 1, 'r'),
    'Carry Rho': cal_partial_derivative(gbsm, 1, 'b')
}

greeks_values = {greek_name: [greek_func(option_type, S, X, T, implied_vol, r, b) for option_type in option_types] for greek_name, greek_func in greek_funcs.items()}

# Modify the Theta values to be negative
greeks_values['Theta'] = [-theta for theta in greeks_values['Theta']]

# Output Greeks values
for greek_name, values in greeks_values.items():
    print(f"{greek_name}:")
    for option_type, value in zip(option_types, values):
        print(f"  {option_type}: {value:.8f}")
    print()


# Finite difference approximation for the Greeks
def finite_difference_greeks(S, K, T, r, q, sigma, option_type='call'):
    h = 1e-5  # Small change for finite differences
    price = black_scholes_price(S, K, T, r, q, sigma, option_type)
    
    # Delta
    price_up = black_scholes_price(S + h, K, T, r, q, sigma, option_type)
    price_down = black_scholes_price(S - h, K, T, r, q, sigma, option_type)
    delta = (price_up - price_down) / (2 * h)
    
    # Gamma
    price_up_up = black_scholes_price(S + 2*h, K, T, r, q, sigma, option_type)
    price_down_down = black_scholes_price(S - 2*h, K, T, r, q, sigma, option_type)
    gamma = (price_up_up - 2*price + price_down_down) / (4 * h**2)
    
    # Theta
    T += 1/365  # Increase by one day
    price_T_up = black_scholes_price(S, K, T, r, q, sigma, option_type)
    theta = (price_T_up - price) / (1/365)
    
    # Vega (change in volatility)
    sigma += h
    price_sigma_up = black_scholes_price(S, K, T, r, q, sigma, option_type)
    vega = (price_sigma_up - price) / (h * 100)  # Presented in per 1% change in volatility
    
    # Rho (change in risk-free rate)
    r += h
    price_r_up = black_scholes_price(S, K, T, r, q, sigma, option_type)
    rho = (price_r_up - price) / (h * 100)  # Presented in per 1% change in risk-free rate
    
    return {
        'price': price,
        'delta': delta,
        'gamma': gamma,
        'theta': theta,
        'vega': vega,
        'rho': rho
    }

# Calculate finite difference Greeks for call and put options
fd_greeks_call = finite_difference_greeks(current_stock_price, strike_price, time_to_expiration, risk_free_rate, continuously_compounding_coupon, volatility, 'call')
fd_greeks_put = finite_difference_greeks(current_stock_price, strike_price, time_to_expiration, risk_free_rate, continuously_compounding_coupon, volatility, 'put')


Delta:
  Call: 0.08297130
  Put: -0.91654963

Gamma:
  Call: 0.01682291
  Put: 0.01682295

Vega:
  Call: 6.93865306
  Put: 6.93865306

Theta:
  Call: -8.12652236
  Put: -1.94099148

Rho:
  Call: -0.03035991
  Put: -1.24273132

Carry Rho:
  Call: 1.13295501
  Put: -12.51527063



({'price': 0.3357989976315192,
  'delta': 0.08297130307255429,
  'gamma': 0.016817658377021868,
  'theta': 8.201791134286553,
  'vega': 22.543363068695754,
  'rho': 22.5552079749729},
 {'price': 13.745361593880062,
  'delta': -0.9165496337004696,
  'gamma': 0.01691091711109038,
  'theta': 2.016661124660999,
  'vega': 5.597801398522506,
  'rho': 5.456555021225995})

# Implement the binomial tree valuation for American options with and without discrete dividends. Assume the stock above: Pays dividend on 4/11/2022 of $0.88

In [3]:
import numpy as np

def binomial_tree_american(S, K, T, r, sigma, n, option_type, dividend_amount=0, dividend_dates=[]):
    # Time step
    dt = T / n
    # Calculate up and down factors
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    # Risk-neutral probability
    p = (np.exp((r - dividend_amount) * dt) - d) / (u - d)

    # Initialize arrays for stock prices and option values
    stock_prices = np.zeros((n+1, n+1))
    option_values = np.zeros((n+1, n+1))

    # Set up stock prices in the tree
    for i in range(n+1):
        for j in range(i+1):
            stock_prices[j, i] = S * (u ** (i-j)) * (d ** j)

    # Set up the last column with payoff at expiration
    for j in range(n+1):
        if option_type == 'call':
            option_values[j, n] = max(0, stock_prices[j, n] - K)
        else:  # put
            option_values[j, n] = max(0, K - stock_prices[j, n])

    # Calculate the option price at each node
    for i in range(n-1, -1, -1):
        for j in range(i+1):
            early_exercise = 0
            if option_type == 'call':
                early_exercise = max(0, stock_prices[j, i] - K)
            else:  # put
                early_exercise = max(0, K - stock_prices[j, i])

            # Adjust for dividends by reducing stock prices at ex-dividend dates
            if dividend_dates and i * dt < dividend_dates[0] <= (i+1) * dt:
                stock_prices[j, i] -= dividend_amount

            # Value of holding the option
            holding_value = (p * option_values[j, i+1] + (1 - p) * option_values[j+1, i+1]) * np.exp(-r * dt)

            # Option value is max of holding and exercising
            option_values[j, i] = max(early_exercise, holding_value)

    return option_values[0, 0]

# Parameters
S = 151.03  # Current stock price
K = 165  # Strike price
# Convert the time difference to float representing the number of days
current_date = np.datetime64('2022-03-13')
expiration_date = np.datetime64('2022-04-15')
T = (expiration_date - current_date).astype('timedelta64[D]').astype(int) / 365# Convert to fraction of year
r = 0.0425  # Risk-free rate
sigma = 0.80  # Volatility
n = 100  # Number of steps in the binomial tree
dividend_amount = 0.88  # Dividend amount
dividend_date = np.datetime64('2022-04-11').astype(int)  # Dividend payment date

# Calculate option values with dividend
call_value_with_dividend = binomial_tree_american(S, K, T, r, sigma, n, 'call', dividend_amount, [dividend_date])
put_value_with_dividend = binomial_tree_american(S, K, T, r, sigma, n, 'put', dividend_amount, [dividend_date])

# Calculate option values without dividend
call_value_no_dividend = binomial_tree_american(S, K, T, r, sigma, n, 'call', 0, [])
put_value_no_dividend = binomial_tree_american(S, K, T, r, sigma, n, 'put', 0, [])

def n_nodes(N):
    return (N + 2) * (N + 1) // 2

def node_index(i, j):
    return n_nodes(j - 1) + i

def binomial_tree_no_div(option_type, S0, X, T, implied_vol, r, N):
    is_call = 1 if option_type == "Call" else -1
    dt = T / N
    disc = np.exp(-r * dt)
    u = np.exp(implied_vol * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    C = np.empty(n_nodes(N), dtype=float)
            
    for i in np.arange(N, -1, -1):
        for j in range(i, -1, -1):
            S = S0 * u ** j * d ** (i - j)
            index = node_index(j, i)
            C[index] = max(0, (S - X) * is_call)
            if i < N:
                val = disc * (p * C[node_index(j + 1, i + 1)] + (1 - p) * C[node_index(j, i + 1)])
                C[index] = max(C[index], val)
                
    return C[0]

def binomial_tree(option_type, S0, X, T, div_time, div, implied_vol, r, N):
    if div_time is None or div is None:
        return binomial_tree_no_div(option_type, S0, X, T, implied_vol, r, N)
  
    is_call = 1 if option_type == "Call" else -1
    dt = T / N
    disc = np.exp(-r * dt)
    
    #calculate u, d, and p
    u = np.exp(implied_vol * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    new_T = T - div_time * dt
    new_N = N - div_time

    C = np.empty(n_nodes(div_time), dtype=float)
    for i in range(div_time, -1, -1):
        for j in range(i, -1, -1):
            S = S0 * u ** j * d ** (i - j)
            val_exe = max(0, (S - X) * is_call)
            if i < div_time:
                val = disc * (p * C[node_index(j + 1, i + 1)] + (1 - p) * C[node_index(j, i + 1)])
            else:
                val = binomial_tree(option_type, S - div, X, new_T, None, None, implied_vol, r, new_N)
            C[node_index(j, i)] = max(val_exe, val)
    
    return C[0]
(call_value_with_dividend, put_value_with_dividend, call_value_no_dividend, put_value_no_dividend)


(5.943325900144015, 30.226821632852445, 9.349599724132323, 22.756696757667296)

# Calculate the Greeks of each

In [4]:
def binomial_tree_greeks(S, K, T, r, sigma, n, option_type, dividend_amount, dividend_dates):
    # Calculate the base option price
    base_price = binomial_tree_american(S, K, T, r, sigma, n, option_type, dividend_amount, dividend_dates)
    
    # Delta and Gamma
    dS = S * 0.01
    price_up = binomial_tree_american(S + dS, K, T, r, sigma, n, option_type, dividend_amount, dividend_dates)
    price_down = binomial_tree_american(S - dS, K, T, r, sigma, n, option_type, dividend_amount, dividend_dates)
    delta = (price_up - price_down) / (2 * dS)
    gamma = (price_up - 2 * base_price + price_down) / (dS ** 2)
    
    # Theta
    dt = 1/365
    price_tomorrow = binomial_tree_american(S, K, T - dt, r, sigma, n, option_type, dividend_amount, dividend_dates)
    theta = (price_tomorrow - base_price) / dt
    
    # Vega
    dvol = sigma * 0.01
    price_vol_up = binomial_tree_american(S, K, T, r, sigma + dvol, n, option_type, dividend_amount, dividend_dates)
    vega = (price_vol_up - base_price) / dvol
    
    # Rho
    dr = 0.0001
    price_r_up = binomial_tree_american(S, K, T, r + dr, sigma, n, option_type, dividend_amount, dividend_dates)
    rho = (price_r_up - base_price) / dr
    
    return {
        'price': base_price,
        'delta': delta,
        'gamma': gamma,
        'theta': theta,
        'vega': vega,
        'rho': rho
    }

# Calculate Greeks for call and put with and without dividends
greeks_call_with_dividend = binomial_tree_greeks(S, K, T, r, sigma, n, 'call', dividend_amount, [dividend_date])
greeks_put_with_dividend = binomial_tree_greeks(S, K, T, r, sigma, n, 'put', dividend_amount, [dividend_date])

greeks_call_no_dividend = binomial_tree_greeks(S, K, T, r, sigma, n, 'call', 0, [])
greeks_put_no_dividend = binomial_tree_greeks(S, K, T, r, sigma, n, 'put', 0, [])

(greeks_call_with_dividend, greeks_put_with_dividend, greeks_call_no_dividend, greeks_put_no_dividend)


({'price': 5.943325900144015,
  'delta': 0.31491723133259686,
  'gamma': 0.008035264186402614,
  'theta': -40.664020637951275,
  'vega': 15.300335670813347,
  'rho': 2.6123541553424445},
 {'price': 30.226821632852445,
  'delta': -0.6439305735493484,
  'gamma': 0.009297238052048256,
  'theta': -147.55478851498364,
  'vega': 14.673856245393413,
  'rho': -11.72046488687073},
 {'price': 9.349599724132323,
  'delta': 0.42641967654359736,
  'gamma': 0.011603243083945261,
  'theta': -82.86069401391454,
  'vega': 18.08344348373936,
  'rho': 4.728038087513653},
 {'price': 22.756696757667296,
  'delta': -0.577471616786322,
  'gamma': 0.013017980071740421,
  'theta': -76.51746265375296,
  'vega': 18.024328802098033,
  'rho': -8.188856681350387})

# What is the sensitivity of the put and call to a change in the dividend amount?

In [6]:
def dividend_sensitivity(S, K, T, r, sigma, n, option_type, dividend_amount, dividend_dates, ddiv=0.01):
    # Base option price with original dividend
    base_price = binomial_tree_american(S, K, T, r, sigma, n, option_type, dividend_amount, dividend_dates)
    
    # Option price with increased dividend
    increased_dividend_price = binomial_tree_american(S, K, T, r, sigma, n, option_type, dividend_amount + ddiv, dividend_dates)
    
    # Sensitivity (first-order derivative approximation)
    sensitivity = (increased_dividend_price - base_price) / ddiv
    
    return sensitivity

# Calculate sensitivity to dividend amount for call and put options
call_dividend_sensitivity = dividend_sensitivity(S, K, T, r, sigma, n, 'call', dividend_amount, [dividend_date])
put_dividend_sensitivity = dividend_sensitivity(S, K, T, r, sigma, n, 'put', dividend_amount, [dividend_date])

(call_dividend_sensitivity, put_dividend_sensitivity)


(-2.9414166825542765, 8.99173044723618)

The result (-2.9414166825542765, 8.99173044723618) represents the sensitivity of the call and put option prices, respectively, to a change in the dividend amount. Here's what each number signifies:

Call Option Sensitivity: -2.9414
This means that for a one-unit increase in the dividend amount, the price of the call option is expected to decrease by approximately 2.9414 units. Call options are generally negatively affected by an increase in dividends because as dividends increase, the expected future stock price decreases (since cash is being taken out of the company and paid to shareholders), making the call option less valuable.

Put Option Sensitivity: 8.9917
This indicates that for a one-unit increase in the dividend amount, the price of the put option is expected to increase by approximately 8.9917 units. Put options tend to become more valuable with an increase in dividends for the opposite reason that call options decrease in value. As the expected future stock price drops due to higher dividends, the put option, which profits from a decrease in the stock price, becomes more valuable.

The sensitivities are quite significant, which may be due to several factors such as the proximity of the option's expiration date to the dividend date, the size of the dividend relative to the stock price, or the volatility of the stock. If the options are close to the money (where the stock price is close to the strike price), even small changes in expected future stock prices due to dividends can have a large impact on the value of the options.